# Directional movement analysis

In [1]:
%load_ext memory_profiler

In [2]:
import pandas as pd 
import numpy as np
import time
%memit np.zeros(100000)
%timeit np.zeros(100000)

peak memory: 98.89 MiB, increment: 0.95 MiB
13.2 µs ± 70.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [3]:
#load data from book J.Welles Wilder
date = range(1,43)
high = [274,273.25,272,270.75,270,270.5,268.5,265.5,262.5,263.5,269.5,267.25,267.5,269.75,268.25,264,268,266,274,277.5,277,272,267.25,269.25,266,265,264.75,261,257.5,259,259.75,257.25,250,254.25,254,253.25,253.25,251.75,253,251.5,246.25,244.25]
low = [272,270.25,269.75,268,269,268,266.5,263,259,260,263,265,265.5,266,263.25,261.5,266.25,264.25,267,273.5,272.5,269.5,264,263,263.5,262,261.5,255.5,253,254,257.5,250,247,252.75,250.50,250.25,251,250.50,249.5,245.25,240,241.25]
close = [272.75,270.75,270,269.25,269.75,270,266.5,263.25,260.25,263,266.5,267,265.75,268.5,264.25,264,266.5,265.25,273,276.75,273,270.25,266.75,263,265.5,262.25,262.75,255.5,253,257.5,257.5,250,249.75,253.75,251.25,250.5,253,251.5,250,245.75,242.75,243.5]

data = {'date': date, 'high': high, 'low':low, 'close': close}

df = pd.DataFrame(data)
print(f"Dataframe shape:{df.shape}")
df

Dataframe shape:(42, 4)


,date,high,low,close
0,1,274.00,272.00,272.75
1,2,273.25,270.25,270.75
2,3,272.00,269.75,270.00
3,4,270.75,268.00,269.25
4,5,270.00,269.00,269.75
5,6,270.50,268.00,270.00
6,7,268.50,266.50,266.50
7,8,265.50,263.00,263.25
8,9,262.50,259.00,260.25
9,10,263.50,260.00,263.00


In [38]:
#load data from csv file
#df = pd.read_csv("data/input/USDT_BTC_86400_1425168000_72.csv")
df = pd.read_csv("data/input/USDT_BTC_300_1425168000_72.csv")
print(f"Dataframe shape:{df.shape}")
df = df.drop(columns=['ts','period','currency_pair','volume','quoteVolume','quoteVolume','weightedAverage'])
df.head()


Dataframe shape:(631047, 11)


,date,high,low,open,close
0,1425168000,240.00006,240.00006,240.00006,240.00006
1,1425168300,240.00006,240.00006,240.00006,240.00006
2,1425168600,240.00006,240.00006,240.00006,240.00006
3,1425168900,240.00006,240.00006,240.00006,240.00006
4,1425169200,240.00006,240.00006,240.00006,240.00006


In [5]:
def true_range(df):
    ranges = [df.high - df.low,(df.high - df.close.shift(1)).abs(),(df.low - df.close.shift(1)).abs()]
    return np.maximum.reduce(ranges)
  

In [6]:
def directional_movement(df):
    delta_high = df.high - df.high.shift(1)
    delta_low = df.low.shift(1) - df.low
    delta_high = np.where(delta_high < 0, 0, delta_high)
    delta_low = np.where(delta_low < 0, 0, delta_low)
    up_dm = np.where(delta_high > delta_low, delta_high, 0)
    down_dm = np.where(delta_low > delta_high, delta_low, 0)
    return up_dm, down_dm
    

In [ ]:
def average_true_range_1(tr,n):
    ATR = np.zeros(tr.size)
    ATR[ATR.index[n-1]] = np.mean(tr[:n])
    
        
    

In [7]:
def average_directional_movement_index_v1(df,n):
    df['tr'] = true_range(df)
    #up_dm, down_dm = directional_movement(df)
    #df['up_dm'] = up_dm
    #df['down_dm'] = down_dm
    #%timeit directional_movement(df) x3 faster
    #%timeit directional_movement2(df)
    up_dm, down_dm = directional_movement(df)
    df['up_dm'] = up_dm
    df['down_dm'] = down_dm
    


In [8]:
average_directional_movement_index_v1(df,14)
df

,date,high,low,close,tr,up_dm,down_dm
0,1,274.00,272.00,272.75,NaN,0.00,0.00
1,2,273.25,270.25,270.75,3.00,0.00,1.75
2,3,272.00,269.75,270.00,2.25,0.00,0.50
3,4,270.75,268.00,269.25,2.75,0.00,1.75
4,5,270.00,269.00,269.75,1.00,0.00,0.00
5,6,270.50,268.00,270.00,2.50,0.00,1.00
6,7,268.50,266.50,266.50,3.50,0.00,1.50
7,8,265.50,263.00,263.25,3.50,0.00,3.50
8,9,262.50,259.00,260.25,4.25,0.00,4.00
9,10,263.50,260.00,263.00,3.50,1.00,0.00
